In [ ]:
import os
# coding: utf-8
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="2" # change 0  with whatever card is available
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
import numpy as np
# from trainVAE_D import trainVAE_D
# coding: utf-8
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import random 
from Constant import Constants
from load_data import StyleData
from PreTrainDs import indexData2variable
import time
torch.set_default_tensor_type(torch.cuda.FloatTensor)


In [ ]:
ds = torch.load('./Model/Ds.pkl').cuda()
ds_emb = torch.load('./Model/Ds_emb.pkl').cuda()

In [ ]:
data = np.load('./data/trainDataOfIndex.npy')

In [ ]:
gan_path = './Model/gan2.pkl'
style_path = './data/style.npy'
epoches = 30
batch_size = 1
pretrainD = True

# trainVAE_D(epoches, batch_size, train_data, ds, ds_emb,  gan_path, style_path,pretrainD)

print ("finished trainning.......................")

In [ ]:
style_data = np.load(style_path)
ind_sentense = data[0][0]
sentense = ' '.join([style_data[1][x] for x in ind_sentense])
sentense

In [ ]:
def unembed(x1, W_normalized, style_data):
    #calculate cosine similarity
    x1_normalized = x1.div(x1.norm(p = 2, dim=1, keepdim=True))
    emb_distances = torch.mm(x1_normalized, W_normalized.t())
    token_ids = torch.argmax(emb_distances, dim=1).cpu().numpy()
    sentense = ' '.join([style_data[1][x] for x in token_ids])
    return (token_ids, sentense)

def ind_to_words(ind_sent, style_data):
    return (' '.join([style_data[1][x] for x in ind_sent]))

def onehot_to_words(onehot, style_data):
    ind_sent = onehot.argmax(dim=1).cpu().numpy()
    print(onehot)
    print(ind_sent)
    return (' '.join([style_data[1][x] for x in ind_sent]))
                
def build2pairs(train_data):
    data = []
    for i in range(min( len(train_data[0]), len(train_data[1]) )):
           data.append([train_data[0][i], train_data[1][i]])
    return data

def shuffleData(train_data):
    """
    this function don't need to return any value and the list is changed inplace
    """
    if len(train_data) == 2:
        random.shuffle(train_data[0])
        random.shuffle(train_data[1])
    else:
        random.shuffle(train_data)

        
def get_d_acc(gan, train_data):
    
    acc = 0
    min_len = len(train_data)/100
    train_data = train_data[:min_len]
    for i in range(min_len):
        dic = gan(train_data[i][0],train_data[i][1],Ez_train=False,Ey_train=False,G_train=False,
                              Lcyc=False, Lrec=False, Ldis = False)
        if dic['D_x1_wl'].topk(1)[1].cpu().data.numpy() == 0:
            acc += 1
        if dic['D_x2_hat'].topk(1)[1].cpu().data.numpy() == 1:
            acc += 1
    
    print(("acc:\t\t %.4f" % (acc/(min_len*2.0))))
    return acc/(min_len*2.0)


In [ ]:
gan = torch.load(gan_path)
gan = gan.cuda()

W = gan.embedding.embedding.weight
W_normalized = W.div(W.norm(p = 2, dim=1, keepdim=True))

style = StyleData()
style.load(style_path)
const = Constants(style.n_words)
optimizer = optim.Adam(gan.parameters(),lr=const.Lr)
lamda1 = 1
lamda2 = 1
lamda3 = 3
cross_entropy = nn.CrossEntropyLoss()

# init the state of some model
ds.train(False)
ds_emb.train(False)

train_data = indexData2variable(data)
train_data = build2pairs(train_data)
# get list([Var(data[0,i]), Var(data[1,i])])
stime = time.time()        
count = 0
while count < len(train_data):
    tempdata = train_data[count:count+batch_size]

    if tempdata == []:
        break

    count += batch_size
    optimizer.zero_grad()
    Lrec = 0
    Lcyc = 0
    Ldis = 0
    Ladv = 0

    Loss = 0

# before we let the D lead the gradient the D model must be strong enough
    for seqs in tempdata:
        seqs[0] = seqs[0].cuda()
        seqs[1] = seqs[1].cuda()
        print(seqs[0])
        dic = gan(seqs[0],seqs[1],D_train=False)
        print(unembed(dic['x1'], W_normalized, style_data)[1])
        print(onehot_to_words(dic['x1_hat'], style_data))
        print(onehot_to_words(dic['x1_bar'], style_data))

etime = time.time()
print(("cost time \t%.2f mins" % ((etime - stime)/60)))

In [ ]:
print(gan)